# Testing Individual components of the FV HE scheme

In [1]:
import random

from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encryptor import Encryptor
from syft.frameworks.torch.he.fv.decryptor import Decryptor
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType
from syft.frameworks.torch.he.fv.evaluator import Evaluator

## Keygeneration

In [2]:
poly_modulus = 64
bit_sizes= [40]
plain_modulus = 64
ctx = Context(EncryptionParams(poly_modulus, CoeffModulus().create(poly_modulus, bit_sizes), plain_modulus))
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [3]:
# print(len(sk.data))
print('secret key values : ', sk.data)

secret key values :  [[1099511623296, 1, 0, 1, 1099511623296, 1099511623296, 0, 1099511623296, 1099511623296, 1, 0, 1099511623296, 0, 0, 1099511623296, 1, 1099511623296, 1099511623296, 1, 0, 1099511623296, 1099511623296, 1099511623296, 1099511623296, 1, 1, 0, 1099511623296, 1099511623296, 1099511623296, 0, 0, 1, 1, 0, 1099511623296, 1099511623296, 0, 1, 0, 1099511623296, 0, 0, 1, 0, 1099511623296, 1099511623296, 0, 0, 1, 1099511623296, 1099511623296, 0, 1, 1, 1099511623296, 1, 1, 1099511623296, 1099511623296, 1, 0, 1099511623296, 1099511623296]]


In [4]:
# print(pk.data)
# print('public key values : ', pk.data)

## Integer Encoder
Encodes Integer values to Plaintext object

In [5]:
int_encoder = IntegerEncoder(ctx)
ri1 = random.randint(0,10)
ri2 = random.randint(0,10)
ri3 = random.randint(0,10)
pt1 = int_encoder.encode(ri1)
pt2 = int_encoder.encode(ri2)
pt3 = int_encoder.encode(ri3)
print(pt1.data,"   ", pt2.data, "   ", pt3.data)
# print('plaintext data',plaintext.data)

[1, 0, 0, 1]     [0, 0, 1]     [1, 1]


### Decodes back to Integer

In [6]:
print(int_encoder.decode(pt1))
print(int_encoder.decode(pt2))
print(int_encoder.decode(pt3))

9
4
3


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [7]:
encrypter = Encryptor(ctx, pk)

In [8]:
ct1 = encrypter.encrypt(pt1)
ct2 = encrypter.encrypt(pt2)
ct3 = encrypter.encrypt(pt3)

Encrypt Plaintext to ciphertext using secret_key

## Decryptor
Decrypt Ciphertext to Plaintext using secret_key

In [9]:
decrypter = Decryptor(ctx, sk)

In [10]:
dec1 = decrypter.decrypt(ct1)
dec2 = decrypter.decrypt(ct2)
dec3 = decrypter.decrypt(ct3)

In [11]:
print(int_encoder.decode(dec1), "   ", int_encoder.decode(dec2), "   ", int_encoder.decode(dec3))

9     4     3


## Evaluator

In [12]:
eval = Evaluator(ctx)

In [13]:
cc12 = eval.add(ct1, ct2)
cc12 = decrypter.decrypt(cc12)
print(int_encoder.decode(cc12))

13


In [14]:
pc12 = eval.add(pt1, ct2)
pc12 = decrypter.decrypt(pc12)
print(int_encoder.decode(pc12))

13


In [15]:
pp12 = eval.add(pt1, pt2)
print(int_encoder.decode(pp12))

13


### Verify result

In [16]:
assert int_encoder.decode(cc12) == int_encoder.decode(pc12) == int_encoder.decode(pp12) == ri1+ri2

In [17]:
result = eval._mul_cipher_cipher(ct1, ct2)
print("\n\nct1 :",ct1.data)
print("\n\nct2 :",ct2.data)
print('\n\n')

result = decrypter.decrypt(result)
result = int_encoder.decode(result)

print('final result: ', result)



ct1 : [[[648049015750, 1043484709929, 912920988260, 293660945893, 55958470559, 1053493554736, 1060064162234, 495328759917, 400442215239, 714769375226, 862549126058, 839009648988, 273167933563, 1041210039819, 104565769485, 438672021277, 227354486880, 919743863436, 67655801645, 745149580147, 684623190479, 326418050213, 550516647939, 366711540046, 678923148900, 243021516943, 113347178801, 773747108623, 21046030315, 790486746825, 414374573822, 107734282631, 537393651736, 1037625753981, 553072713228, 156202043248, 864642632819, 55967711428, 673327214613, 475722963085, 350058474501, 615186490126, 322254020297, 240863653461, 1008076811916, 1072674176092, 188725472653, 650020438889, 500910878724, 327360050390, 160148695921, 988458348230, 795149157322, 50990382414, 123027322620, 685426358920, 1035535730311, 765571504981, 912164996823, 1089916865282, 224591250551, 647825835704, 688663252270, 208494896197]], [[489013322621, 20686660579, 723978460245, 648958234798, 976321953659, 55343303408, 233

In [18]:
print(ri1 * ri2, "    ", result)
assert ri1 * ri2 == result

36      36


## Tensor Testing

In [19]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
context = Context(EncryptionParams(128, CoeffModulus().create(64, [30, 30]), 64))

In [20]:
key_generator = KeyGenerator(context)
keys = key_generator.keygen()
relin_key = key_generator.get_relin_keys()

In [21]:
x = th.Tensor([[1,2,3],[4,5,6]]).int().encrypt(protocol="bfv", private_key=keys[0], context = context)

In [22]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x123f91f40>

In [23]:
y = x.decrypt(private_key = keys[0])
y

tensor([[1, 2, 3],
        [4, 5, 6]])

In [24]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x123f91f40>

In [25]:
x = x + y

In [26]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x1243224c0>

In [27]:
x.decrypt(private_key = keys[0])

tensor([[ 2,  4,  6],
        [ 8, 10, 12]])

In [28]:
x = x - th.Tensor([[1,2,3],[1,2,3]]).int().encrypt(protocol="bfv", private_key=keys[0], context = context)

In [29]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x1243450a0>

In [30]:
x.decrypt(private_key = keys[0])

tensor([[1, 2, 3],
        [7, 8, 9]])

In [31]:
 x = x*x

In [32]:
x.decrypt(private_key = keys[0])

tensor([[ 1,  4,  9],
        [49, 64, 81]])

In [33]:
a = th.Tensor([[1,2,3],[4,5,6]]).int().encrypt(protocol="bfv", private_key=keys[0], context = context)
b = th.Tensor([[-1,-2],[-3,-4], [-5,-6]]).int().encrypt(protocol="bfv", private_key=keys[0], context = context)

In [34]:
a

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x123f66b80>

In [35]:
b

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x1242df3d0>

In [36]:
ans_mm = a.mm(b, relin_key = relin_key)
print(ans_mm)

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12435d670>


In [37]:
mm_decrypt = ans_mm.decrypt(private_key = keys[0])

In [38]:
print(mm_decrypt)

tensor([[-22, -28],
        [-49, -64]])
